In [1]:
#BIBLIOTECAS USADAS

from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
%run database.ipynb 


2024-02-24 07:03:39,655 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-24 07:03:39,657 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("prod_venda")
2024-02-24 07:03:39,659 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-24 07:03:39,662 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("produto")
2024-02-24 07:03:39,663 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-24 07:03:39,665 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("venda")
2024-02-24 07:03:39,666 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-24 07:03:39,668 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("usuario")
2024-02-24 07:03:39,668 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-24 07:03:39,670 INFO sqlalchemy.engine.Engine COMMIT


In [2]:
### FUNÇÕES DE VERIFICAÇÃO ###

def produto_existente(session, nome_produto):
    return session.query(Produto).filter_by(name=nome_produto).first() is not None

def verificar_disponibilidade_produto(session, nome_produto, quantidade_pedida):
    produto = session.query(Produto).filter_by(name=nome_produto).first()

    if produto and produto.quantity >= quantidade_pedida:
        return True
    else:
        return False

In [3]:
### FUNÇÃO ENTRADA DE PRODUTOS ###

def entrada_produtos(session):
    novo_produto = input('Digite o nome do novo produto: ').casefold()
    
    if not produto_existente(session, novo_produto):
        estoque = int(input(f'Digite a quantidade de estoque do produto {novo_produto}: '))
        preco = float(input(f'Digite o preço do produto {novo_produto}: '))
        
        novo_produto = Produto(name=novo_produto, quantity=estoque, price=preco)
        session.add(novo_produto)
        session.commit()
        
        try:
            session.add(novo_produto)
            session.commit()
            print(f"Produto '{novo_produto.name}' adicionado com sucesso!")
        except Exception as e:
            session.rollback()
            print(f"Erro ao adicionar o produto: {e}")
        
    else:
        print(f"Produto '{novo_produto}' já existe na tabela de produtos. Adicione outro novamente.")

# Exemplo de uso da função

#Pedir pro usuário inserir novo produto (Produto, quantidade no estoque e preço)
#Fazer tratamento de casefold() no nome do produto
#Verificar no BD se já existe este produto no BD!
    #Caso exista, pedir pro usuário inserir novo produto novamente
    #Caso não exista, inserir no BD o produto com estoque + mostrar msg pro usuário

In [4]:
### FUNÇÃO DE LISTAGEM DOS PRODUTOS ###

def listar_produtos (session):
    produtos = session.query(Produto).all()

    if not produtos:
        print("Não há produtos cadastrados.")
    else:
        print("Lista de Produtos:")
        for produto in produtos:
            print(f"ID: {produto.id_produto}, Nome: {produto.name}, Quantidade: {produto.quantity}, Preço: {produto.price}")

In [5]:
### FUNÇÃO VENDA ###

def venda_produtos (session):
    listar_produtos(session)
    valor_total_compra = 0
    while(True):
        compra_produto = input('Digite qual produto deseja comprar (Para sair digite "sair"):')

        if compra_produto == 'sair':
            break

        else: 
            qtd_pedida = int(input('Digite qual é a quantidade do produto {compra_produto} que deseja'))
            if not verificar_disponibilidade_produto(session, compra_produto, qtd_pedida):
                print('Não há disponibilidade suficiente de produto')
            else:
                produto = session.query(Produto).filter_by(name=compra_produto).first()
                produto.quantity -= qtd_pedida
                valor_total_compra += (produto.price * qtd_pedida)

    # Criar uma instância de Venda após o loop de seleção de produtos
    venda = Venda(valor_total = valor_total_compra)
    # Adicionar a venda ao banco de dados
    session.add(venda)
    session.commit()

    print('O VALOR DA SUA COMPRA FOI DE R${valor_total_compra}')

    #Retornar para o usuário a listagem com produtos e preços
    #Lista de compras (Repetição - While(True)) 
        #Qual produto usuário deseja comprar e qual a quantidade 
        #Verificar no BD se tem o produto e se tem quantidade no estoque
            #Caso não tenha o produto ou quantidade no estoque, retornar pro usuário msg 
            #Caso tenha, acrescentar em um dicionario as informações do nome do produto + valor_total_parcial
        #Perguntar ao usuário se deseja comprar mais algum produto ou sair
            #Repetir todo processo e somar no valor_total_parcial
    #Pegar o dicionario criado, percorrer para mostrar pro usuário os produtos que ele comprou
        #Colocar na tabela venda o valor total

In [8]:
### FUNÇÃO CADASTRO USUÁRIO ###

def cadastro_usuario (session):
    usuarios = session.query(Usuario).all()
    print('\n\n========= LOGIN ===========\n\n')
    op = int(input('Cadastro Usuário (1) | Login (2)'))
    
    ###OPÇÃO DE CADASTRO DO USUÁRIO###
    if op == 1:
        cpf_usuario = input('Digite o usuario (CPF): ')
        nome = input ('Digite seu nome: ')
        senha = input('Digite a senha: ')

        if cpf_usuario not in [usuario.cpf_usuario for usuario in usuarios]:
            novo_usuario = Usuario(cpf_usuario=cpf_usuario, nome_usuario=nome, senha=senha)
            try:
                session.add(novo_usuario)
                session.commit()
                print(f"Usuário '{novo_usuario.nome_usuario}' cadastrado com sucesso!")
            except Exception as e:
                session.rollback()
                print(f"Erro ao cadastrar usuário: {e}")

        else:
            print('Sistema já possui usuário cadastrado com este CPF. Por favor, cadastre novamente')

    ###OPÇÃO DE LOGIN DO USUÁRIO###
    if op == 2:
        cpf_consulta = input('Digite seu CPF para Login: ')
        senha_consulta = input('Insira sua senha: ')

        usuario = session.query(Usuario).filter_by(cpf_usuario=cpf_consulta).first()

        if usuario:
            if usuario.senha == senha_consulta:
                print(f'Bem-vindo, {usuario.nome_usuario}!')
            else:
                print('Senha incorreta. Tente novamente.')
        else:
            print('Não possui este cadastro. Por favor, faça um cadastro')

In [7]:
### FUNÇÃO MAIN ###

def main():
    Session = sessionmaker(bind=engine)
    session = Session()


    while(True):
        print('\n\n SISTEMA DE GERENCIAMENTO DE PADARIA:')
        print('1- Cadastro de Usuário:')
        print('2- Cadastro de Produto:')
        print('3- Listagem de Produtos')
        print('4- Venda de Produtos')

        opcao = input('Qual opção vc deseja? (Digite "sair" para sair)')
        if opcao == '1':
            cadastro_usuario(session)
        if opcao == '2':
            entrada_produtos(session)
        elif opcao == '3':
            listar_produtos(session)
        elif opcao == '4':
            venda_produtos(session)
        elif opcao == 'sair':
            break

